In [1]:
from IPython.display import display, HTML

display(HTML(data="""

<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

%reload_ext jupyternotify

#%notify

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import sys
from qutip import *
import matplotlib.pyplot as plt
from pathlib import Path

from scipy import linalg as la

parent_path = ".."  #str(Path(__file__).resolve().parents[1])
sys.path.append(parent_path)

import pysqkit


import time
import os

from pysqkit.solvers.solvkit import integrate

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10, 7)

In [3]:
from math import pi

h = 6.62607 * 10**(-34)
h_bar = h/(2*pi)
e = 1.6071766*10**(-19)
phi_0 = h/(2*e)



In [4]:
N = 100

# Init

In [5]:
J_c = 300*10**6 *h
lam = .5

In [6]:
t = np.linspace(0, 400, 2000) * 10**(-9) #in nano seconds

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

## flxA



$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

In [7]:
e_c_A = 1*10**9 *h
e_l_A = 1.5*10**9 *h
e_j_A = 3.8*10**9 *h
  
flux_A = 1/2  # 1/2 -> pi dans le cos

flxA = pysqkit.qubits.Fluxonium(label = "QBA",
                               charge_energy=e_c_A,
                               induct_energy=e_l_A,
                               joseph_energy=e_j_A,
                               flux=flux_A,
                               basis=None,
                               dim_hilbert=N)

In [9]:
eig_enegies, eig_states = flxA.eig_states(levels=5)
flxA.basis.transform(eig_states)

ValueError: The transformaton matrix should be a 2D square matrix, with the same dimensionality as the basis, instead got a 2 dimensional matrix

In [10]:
#Bare energies

bare_E_A = flxA.eig_energies()
bare_E_A/(h*10**9)

array([ 3.12560598,  4.2772998 ,  7.55777961, 10.81091566, 14.50310188,
       18.35763955, 22.30068932, 26.25193133, 30.14840206, 33.93656752])

In [11]:
# transitions freqs

omega_01_A = bare_E_A[1] - bare_E_A[0]
omega_12_A = bare_E_A[2] - bare_E_A[1]

omega_01_A/(h*10**9) , omega_12_A/(h*10**9) 

(1.1516938223021156, 3.2804798038301763)

In [12]:
# mat elements :

n_10_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxA.mat_elements('charge_op', levels = 5)).data, 2), n_01_A, n_10_A

(array([[0.  , 0.25, 0.  , 0.26, 0.  ],
        [0.25, 0.  , 0.61, 0.  , 0.18],
        [0.  , 0.61, 0.  , 0.75, 0.  ],
        [0.26, 0.  , 0.75, 0.  , 0.94],
        [0.  , 0.18, 0.  , 0.94, 0.  ]]),
 0.24866384335580308,
 0.24866384335580308)

### Rq)

In [ ]:
plt.figure()
for i in range(len(bare_E_A)):
    plt.scatter(i, bare_E_A[i])

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.charge_zpf*(create(100) - destroy(100))*1j

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.hamiltonian(as_qobj = True)/(h*10**9)

## flxB

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

In [13]:
e_c_B = 1*10**9 *h
e_l_B = .9*10**9 *h
e_j_B = 3*10**9 *h

flux_B = 1/2

flxB = pysqkit.qubits.Fluxonium(label = "QBB",
                               charge_energy=e_c_B,
                               induct_energy=e_l_B,
                               joseph_energy=e_j_B,
                               flux=flux_B,
                               basis=None,
                               dim_hilbert=N)

In [14]:
eig_enegies, eig_states = flxB.eig_states(levels=5)
flxB.basis.transform(eig_states)

ValueError: The transformaton matrix should be a 2D square matrix, with the same dimensionality as the basis, instead got a 2 dimensional matrix

In [15]:
#Bare energies

bare_E_B = flxB.eig_energies()
bare_E_B/(h*10**9)

array([ 2.07847477,  2.92730143,  5.85594017,  8.53943723, 11.62680359,
       14.77621367, 17.91620664, 20.9451284 , 23.78622957, 26.398564  ])

In [16]:
# transitions freqs

omega_01_B = bare_E_B[1] - bare_E_B[0]
omega_12_B = bare_E_B[2] - bare_E_B[1]

omega_01_B/(h*10**9) , omega_12_B/(h*10**9) 

(0.8488266605312955, 2.928638737997042)

In [17]:
# mat elements :

n_10_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxB.mat_elements('charge_op', levels = 5)).data, 2), n_01_B, n_10_B

(array([[0.  , 0.21, 0.  , 0.28, 0.  ],
        [0.21, 0.  , 0.57, 0.  , 0.17],
        [0.  , 0.57, 0.  , 0.67, 0.  ],
        [0.28, 0.  , 0.67, 0.  , 0.86],
        [0.  , 0.17, 0.  , 0.86, 0.  ]]),
 0.2072248733629435,
 0.20722487336294343)

## coupling


$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

In [18]:
g = J_c#J_c/(4*e**2)

In [19]:
duo = flxA.couple_to(flxB, pysqkit.couplers.capacitive_coupling, strength = g)

In [20]:
duo.eig_energies()/(h*10**9)

array([ 5.20384119,  6.0514322 ,  6.35584493,  7.20120898,  8.98118327,
        9.63508529, 10.10111732, 10.5095348 , 11.66444175, 12.7870574 ])

In [21]:
#by default, restricts only to 6 first levels of each qubit -> 36x36 hamiltonians !

trunc_levels = {
    'QBA': 5 ,
    'QBB': 5    
}

In [22]:
H_duo_bare = duo.bare_hamiltonian(truncated_levels = trunc_levels)
H_duo_bare.shape

(25, 25)

In [23]:
H_duo_int = duo.int_hamiltonian(truncated_levels = trunc_levels)
H_duo_int.shape

(25, 25)

In [24]:
H_duo_tot = duo.hamiltonian(truncated_levels = trunc_levels)
H_duo_tot.shape

(25, 25)

In [49]:
# hA + hB + V (fixe)
#duo hamil to qobj
duo_hamil = Qobj(
                inpt=H_duo_tot,
                dims=[N, N],
                shape=H_duo_tot.shape,
                type='oper',
                isherm=True)

duo_hamil

Quantum object: dims = [100, 100], shape = (25, 25), type = other
Qobj data =
[[ 3.44826033e-24  2.02720873e-57 -5.39887833e-72 -2.71289158e-57
   2.55966030e-72  2.41595644e-57  1.02431091e-26 -2.72795292e-41
  -1.37077372e-26  1.29334880e-41 -1.81642802e-72 -7.70124417e-42
   2.05100144e-56  1.03061121e-41 -9.72399568e-57 -2.52784056e-57
  -1.07174725e-26  2.85428575e-41  1.43425492e-26 -1.35324441e-41
  -3.50095053e-73 -1.48432388e-42  3.95306309e-57  1.98638141e-42
  -1.87418535e-57]
 [-2.02720873e-57  4.01069882e-24  5.54191804e-57  4.83834457e-73
  -1.69283000e-57 -1.02431091e-26  1.25989192e-57  2.80022822e-26
   2.44472562e-42 -8.55355550e-27  7.70124417e-42 -9.47245137e-73
  -2.10534136e-41 -1.83805803e-57  6.43095945e-42  1.07174725e-26
  -1.31823813e-57 -2.92990816e-26 -2.55794206e-42  8.94967484e-27
   1.48432388e-42 -1.82570315e-73 -4.05779687e-42 -3.54263981e-58
   1.23949150e-42]
 [ 5.39887833e-72 -5.54191804e-57  5.95123535e-24 -6.57786722e-57
  -2.62745421e-72  2.72795

### rqs

In [ ]:
duo.bare_hamiltonian(), duo.int_hamiltonian()

In [ ]:
#Bare energies

bare_E_duo = la.eig(duo.bare_hamiltonian())[0].real/h #in Hz
bare_E_duo.sort()
bare_E_duo = bare_E_duo[:5]*10**(-9)  #in GHz

int_E_duo = la.eig(duo.int_hamiltonian())[0].real/h
int_E_duo.sort()
int_E_duo = int_E_duo[:5]*10**(-9)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (10,5))

axs[0].scatter(range(len(bare_E_duo)), bare_E_duo, c = 'r', label = 'bare')    
axs[1].scatter(range(len(int_E_duo)), int_E_duo, c = 'b', label = 'int')

axs[0].legend()
axs[1].legend()

axs[0].set_ylabel("E (GHz)")
axs[1].set_ylabel("E (GHz)")

In [ ]:
E_duo = bare_E_duo + int_E_duo
E_duo.sort()

plt.figure()
plt.scatter(range(len(E_duo)), E_duo)

plt.annotate("  |00>", (0,E_duo[0]))
plt.annotate("  |01>", (1,E_duo[1]))
plt.annotate("  |10>", (2,E_duo[2]))
plt.annotate("  |11>", (3,E_duo[3]))
plt.annotate("  |??>", (4,E_duo[4]))




plt.ylabel('E (GHz)')

In [ ]:
print("Ground =", E_duo[0], "GHz\n1st state =", E_duo[1], "GHz\n2nd state =", E_duo[2], "GHz" )

print("\n\nomega 00->01 =", E_duo[1]-E_duo[0], "GHz\nomega 01->10 =", E_duo[2] - E_duo[1], "GHz\nomega 10->11 =", E_duo[3]-E_duo[2], "GHz"
     )

In [ ]:
duo.hamiltonian()

## Drive

$$\hat{H}_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \cdot \left( \eta_A \cdot \hat{n}_A + \eta_B \cdot \hat{n}_B \right)$$

$$f\left( t \right) = f_0 \cdot \dfrac{f\left( t \right)}{f_0}$$

In [27]:
f_0 = lam/2 * np.abs(omega_01_A - omega_01_B) / (n_10_A)

In [62]:
t_rise = 25*10**(-9)
sigma = t_rise/2

f_by_f0 = np.exp(- (t-t_rise)**2 / (2*sigma**2)) - np.exp(-t_rise**2 /(2*sigma**2))*np.ones(len(t))

In [63]:
f = f_0 * f_by_f0

omega_d = 2*pi * 1 *10**9
gamma_d = 0

eta_1 = 1
eta_2 = 1

In [64]:
n_A = flxA.charge_op(as_qobj = True)
n_B = flxB.charge_op(as_qobj = True)

In [65]:
drive = n_A + n_B

In [66]:
pulse = 2 * f * np.cos(omega_d*t)

### Rq)

In [ ]:
n_1+n_2 #Rq) only different parity term

In [ ]:
n_1*n_2 #Rq) only same parity terms

# Simu

##  With Solver

In [33]:
#duo hamil to qobj
dim = N

duo_hamil = Qobj(
                inpt=H_duo_tot,
                dims=[dim, dim],
                shape=H_duo_tot.shape,
                type='oper',
                isherm=True)

In [38]:
time =  t
state_init = fock(N,0)
hamil = duo_hamil
drive = [drive]
pulse = [pulse]
jump = []
solver = pysqkit.solvers.solvkit.supported_solvers[0]

In [39]:
res = integrate(time, state_init, hamil, drive, pulse, jump, solver)

TypeError: Incorrect Q_object specification

## With mesolve

In [68]:
hamil = [[duo_hamil, np.ones(len(t))], [drive, pulse]]
state_in = fock(25,0)
t_list = t

In [69]:
len(pulse)
pulse

array([ 0.00000000e+00,  5.45740567e-28, -2.90003757e-27, ...,
        4.42211911e-26, -1.68429371e-26, -5.46105570e-26])

In [70]:
res = mesolve(hamil, state_in, t_list)

TypeError: Qobj not compatible.

In [71]:
n_A

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.46530243j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.46530243j 0.+0.j         0.-0.65803701j ... 0.+0.j
  0.+0.j         0.+0.j        ]
 [0.+0.j         0.+0.65803701j 0.+0.j         ... 0.+0.j
  0.+0.j         0.+0.j        ]
 ...
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.-4.60625905j 0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+4.60625905j
  0.+0.j         0.-4.62970072j]
 [0.+0.j         0.+0.j         0.+0.j         ... 0.+0.j
  0.+4.62970072j 0.+0.j        ]]

In [72]:
n_A**2

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[  0.21650635   0.          -0.30618622 ...   0.           0.
    0.        ]
 [  0.           0.64951905   0.         ...   0.           0.
    0.        ]
 [ -0.30618622   0.           1.08253175 ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...  42.21873843   0.
  -21.32560081]
 [  0.           0.           0.         ...   0.          42.65175114
    0.        ]
 [  0.           0.           0.         ... -21.32560081   0.
   21.43412874]]

In [79]:
n_A*n_B

Quantum object: dims = [[100], [100]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[  0.19054978   0.          -0.26947808 ...   0.           0.
    0.        ]
 [  0.           0.57164934   0.         ...   0.           0.
    0.        ]
 [ -0.26947808   0.           0.9527489  ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...  37.15720721   0.
  -18.76891157]
 [  0.           0.           0.         ...   0.          37.53830677
    0.        ]
 [  0.           0.           0.         ... -18.76891157   0.
   18.86442828]]

In [80]:
duo.int_hamiltonian(truncated_levels = {'QBA' :10 , 'QBB' : 10})/J_c

array([[ 9.50679085e-66+0.j,  5.11222423e-33+0.j, -1.40005346e-47+0.j,
        ...,  1.08429544e-05+0.j, -3.18983819e-20+0.j,
        -3.66989002e-06+0.j],
       [-5.11222423e-33+0.j,  4.79852499e-65+0.j,  1.39756342e-32+0.j,
        ...,  8.47972464e-20+0.j, -3.33985592e-07+0.j,
         3.59386802e-19+0.j],
       [ 1.40005346e-47+0.j, -1.39756342e-32+0.j,  2.14519249e-63+0.j,
        ...,  5.76251805e-05+0.j, -1.41866424e-20+0.j,
        -7.26919453e-06+0.j],
       ...,
       [ 1.08429544e-05+0.j,  8.47972464e-20+0.j,  5.76251805e-05+0.j,
        ..., -2.75556779e-63+0.j, -3.31844860e-32+0.j,
         8.76483366e-47+0.j],
       [-3.18983819e-20+0.j, -3.33985592e-07+0.j, -1.41866424e-20+0.j,
        ...,  3.31844860e-32+0.j, -3.66522610e-64+0.j,
        -3.36937394e-32+0.j],
       [-3.66989002e-06+0.j,  3.59386802e-19+0.j, -7.26919453e-06+0.j,
        ..., -8.76483366e-47+0.j,  3.36937394e-32+0.j,
         1.11812785e-64+0.j]])